In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy import linalg as la
from numpy.linalg import matrix_power
import networkx as nx

def checkMatrix(A):
    """
    Checks if the input matrix is a valid square matrix.

    Parameters:
    - A (numpy.ndarray): Input matrix.

    Raises:
    - Exception: If the matrix is not 2-dimensional, not square, or has dimensions less than 2x2.
    """
    if len(A.shape) != 2:
        raise Exception('Matrix is not 2 dimensional')
    if A.shape[0] != A.shape[1]:
        raise Exception('Matrix is not square')
    if A.shape[0] < 2:
        raise Exception('Matrix dimensions should be nxn, n>=2')

def objectiveFunction(A, X, n):
    """
    Creates the objective function for the Traveling Salesman Problem (TSP) relaxation.

    Parameters:
    - A (numpy.ndarray): Cost matrix for TSP.
    - X (cvxpy.Variable): Decision variable matrix.
    - n (int): Size of the matrix.

    Returns:
    - cp.Minimize: Objective function for the TSP relaxation.
    """
    objectiveTerms = []
    for i in range(n):
        for j in range(n):
            objectiveTerms.append(A[i][j] * X[i][j])
    objective = cp.Minimize(sum(objectiveTerms))
    return objective

def exitConstraints(X, n):
    """
    Creates constraints to ensure that each row of the decision variable matrix sums to 1.

    Parameters:
    - X (cvxpy.Variable): Decision variable matrix.
    - n (int): Size of the matrix.

    Returns:
    - list: List of constraints.
    """
    return [sum(X[i]) == 1 for i in range(n)]

def entryConstraints(X, n):
    """
    Creates constraints to ensure that each column of the decision variable matrix sums to 1.

    Parameters:
    - X (cvxpy.Variable): Decision variable matrix.
    - n (int): Size of the matrix.

    Returns:
    - list: List of constraints.
    """
    return [sum(X[:, j]) == 1 for j in range(n)]

def solve(objective, constraints, X):
    """
    Solves the convex optimization problem.

    Parameters:
    - objective (cp.Minimize): Objective function.
    - constraints (list): List of constraints.
    - X (cvxpy.Variable): Decision variable matrix.

    Returns:
    - numpy.ndarray: Solution matrix.
    """
    prob = cp.Problem(objective, constraints)
    prob.solve()
    solution = X.value
    return solution

def findCycles(X):
    """
    Finds cycles in a directed graph represented by the decision variable matrix.

    Parameters:
    - X (numpy.ndarray): Decision variable matrix.

    Returns:
    - list: List of cycles.
    """
    Graph = nx.DiGraph(X)
    simple = nx.simple_cycles(Graph)
    cycles = [list(cycle) for cycle in simple]
    return cycles

def cycleLength(cycles, A):
    """
    Calculates the total length of each cycle in terms of the cost matrix A.

    Parameters:
    - cycles (list): List of cycles.
    - A (numpy.ndarray): Cost matrix.

    Returns:
    - list: List of lengths for each cycle.
    """
    lengths = []
    length = 0
    for cycle in cycles:
        for idx, node in enumerate(cycle):
            if idx == len(cycle) - 1:
                length += A[node][cycle[0]]
                lengths.append(length)
                length = 0
            else:
                length += A[node][cycle[idx + 1]]
    return lengths

def printSolution(cycles, lengths):
    """
    Prints the solution of the TSP relaxation problem.

    Parameters:
    - cycles (list): List of cycles.
    - lengths (list): List of lengths for each cycle.
    """
    for idx, cycle in enumerate(cycles):
        print(f'{idx + 1}. {cycle} with a total length of {lengths[idx]}')
    if len(cycles) == 1:
        print(f'Found a feasible solution to the TSP problem with a value of {lengths[0]}')
    else:
        print(f'Found an infeasible solution to the TSP problem with a value of {sum(lengths)}')

def TSP_relaxation(A):
    """
    Solves the Traveling Salesman Problem (TSP) relaxation for a given cost matrix A.

    Parameters:
    - A (numpy.ndarray): Cost matrix for TSP.
    """
    checkMatrix(A)
    n = len(A)
    X = cp.Variable((n, n), boolean=True)
    objective = objectiveFunction(A, X, n)
    constraints = entryConstraints(X, n) + exitConstraints(X, n)
    solution = solve(objective, constraints, X)
    cycles = findCycles(solution)
    cycle_lengths = cycleLength(cycles, A)
    return (cycles, cycle_lengths)

In [2]:
M = 1000000
A = np.block([[M, 28, 29, 44, 4, 12, 18, 10, 9, 30],
            [31, M, 8, 13, 15, 1, 19, 22, 7, 27],
            [22, 11, M, 40, 29, 32, 4, 13, 10, 17],
            [8, 27, 35, M, 9, 26, 7, 2, 15, 28],
            [28, 43, 17, 1, M, 7, 3, 26, 17, 39],
            [34, 2, 40, 23, 49, M, 47, 28, 10, 19],
            [2, 9, 48, 21, 10, 35, M, 8, 50, 1],
            [47, 35, 5, 38, 39, 11, 7, M, 16, 8],
            [24, 19, 34, 12, 40, 33, 17, 7, M, 30],
            [46, 39, 60, 20, 6, 23, 2, 11, 27, M]])

printSolution(*TSP_relaxation(A))

B = np.block([[M, 28, 29, 44, 4, 12, 18, 10, 9, 30],
            [31, M, 8, 13, 15, 1, 19, 22, 7, 27],
            [22, 11, M, 40, 29, 32, 4, 13, 10, 17],
            [8, 27, 35, M, 9, 26, 7, 2, 15, 28],
            [28, 43, 17, 1, M, 7, 3, 26, 17, 39],
            [34, 2, 40, 23, 49, M, 47, 28, 10, 19],
            [2, 9, 48, 21, 10, 35, M, 8, 50, 1],
            [47, 35, 5, 38, 39, 11, 7, M, 16, 8],
            [24, 19, 34, 12, 40, 33, 17, 7, M, 30],
            [46, 39, 60, 20, 6, 23, M, 11, 27, M]])

printSolution(*TSP_relaxation(B))

C = np.block([[M, 28, 29, 44, 4, 12, 18, 10, 9, 30],
            [31, M, 8, 13, 15, 1, 19, 22, 7, 27],
            [22, 11, M, 40, 29, 32, 4, 13, 10, 17],
            [8, 27, 35, M, 9, 26, 7, 2, 15, 28],
            [28, 43, 17, 1, M, 7, 3, 26, 17, 39],
            [34, 2, 40, 23, 49, M, 47, 28, 10, 19],
            [2, 9, 48, 21, 10, 35, M, 8, 50, M],
            [47, 35, 5, 38, 39, 11, 7, M, 16, 8],
            [24, 19, 34, 12, 40, 33, 17, 7, M, 30],
            [46, 39, 60, 20, 6, 23, 2, 11, 27, M]])
printSolution(*TSP_relaxation(C))

D = np.block([[M, 28, 29, 44, 4, 12, 18, 10, 9, 30],
            [31, M, 8, 13, 15, M, 19, 22, 7, 27],
            [22, 11, M, 40, 29, 32, 4, 13, 10, 17],
            [8, 27, 35, M, 9, 26, 7, 2, 15, 28],
            [28, 43, 17, 1, M, 7, 3, 26, 17, 39],
            [34, 2, 40, 23, 49, M, 47, 28, 10, 19],
            [2, 9, 48, 21, 10, 35, M, 8, 50, 1],
            [47, 35, 5, 38, 39, 11, 7, M, 16, 8],
            [24, 19, 34, 12, 40, 33, 17, 7, M, 30],
            [46, 39, 60, 20, 6, 23, M, 11, 27, M]])

printSolution(*TSP_relaxation(D))

E = np.block([[M, 28, 29, 44, 4, 12, 18, 10, 9, 30],
            [31, M, 8, 13, 15, 1, 19, 22, 7, 27],
            [22, 11, M, 40, 29, 32, 4, 13, 10, 17],
            [8, 27, 35, M, 9, 26, 7, 2, 15, 28],
            [28, 43, 17, 1, M, 7, 3, 26, 17, 39],
            [34, M, 40, 23, 49, M, 47, 28, 10, 19],
            [2, 9, 48, 21, 10, 35, M, 8, 50, 1],
            [47, 35, 5, 38, 39, 11, 7, M, 16, 8],
            [24, 19, 34, 12, 40, 33, 17, 7, M, 30],
            [46, 39, 60, 20, 6, 23, M, 11, 27, M]])

printSolution(*TSP_relaxation(E))



Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-28
1. [9, 6] with a total length of 3
2. [8, 7, 2] with a total length of 22
3. [1, 5] with a total length of 3
4. [0, 4, 3] with a total length of 13
Found an infeasible solution to the TSP problem with a value of 41
1. [1, 5] with a total length of 3
2. [0, 8, 7, 2, 6, 9, 4, 3] with a total length of 41
Found an infeasible solution to the TSP problem with a value of 44
1. [3, 4] with a total length of 10
2. [1, 5] with a total length of 3
3. [0, 8, 7, 2, 9, 6] with a total length of 42
Found an infeasible solution to the TSP problem with a value of 55
1. [0, 5, 1, 8, 7, 2, 6, 9, 4, 3] with a total length of 53
Found a feasible solution to the TSP problem with a value of 53
1. [9, 7, 2, 6] with a total length of 21
2. [8, 1, 5] with a total length of 30
3. [0, 4, 3] with a total length of 13
Found an infeasible solution to the TSP problem with a value of 64
